In [101]:
#importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [102]:
data=pd.read_csv('cpdata.csv')
print(data.head(1))

   temperature   humidity    rainfall label
0    20.879744  82.002744  202.935536  rice


In [103]:
#Creating dummy variable for target i.e label
label= pd.get_dummies(data.label).iloc[: , 1:]
data= pd.concat([data,label],axis=1)
data.drop('label', axis=1,inplace=True)
print('The data present in one row of the dataset is:')
print(data.head(2))
column_headers = list(data.columns.values)
print("The Column Header :", column_headers)
train=data.iloc[:, 0:3].values
test=data.iloc[: ,3:].values

The data present in one row of the dataset is:
   temperature   humidity    rainfall  Black gram  Chickpea  Coconut  Coffee  \
0    20.879744  82.002744  202.935536           0         0        0       0   
1    21.770462  80.319644  226.655537           0         0        0       0   

   Cotton  Ground Nut  Jute  ...  maize  mango  millet  muskmelon  orange  \
0       0           0     0  ...      0      0       0          0       0   
1       0           0     0  ...      0      0       0          0       0   

   papaya  pomegranate  rice  watermelon  wheat  
0       0            0     1           0      0  
1       0            0     1           0      0  

[2 rows x 33 columns]
The Column Header : ['temperature', 'humidity', 'rainfall', 'Black gram', 'Chickpea', 'Coconut', 'Coffee', 'Cotton', 'Ground Nut', 'Jute', 'Kidney Beans', 'Lentil', 'Moth Beans', 'Mung Bean', 'Peas', 'Pigeon Peas', 'Rubber', 'Sugarcane', 'Tea', 'Tobacco', 'apple', 'banana', 'grapes', 'maize', 'mango', 'mil

In [104]:
#Dividing the data into training and test set
X_train,X_test,y_train,y_test=train_test_split(train,test,test_size=0.1)
print(X_train.shape)
print(X_test[:2])
print(y_train.shape)
print(y_test[0])

(2790, 3)
[[28.06642822 85.91625451 39.23831035]
 [21.3272581  50.74277753 86.54737814]]
(2790, 30)
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [105]:
def init_var():
    w1 = np.random.rand(30, 3) - 0.5
    b1 = np.random.rand(30, 1) - 0.5
    w2 = np.random.rand(30, 30) - 0.5
    b2 = np.random.rand(30, 1) - 0.5
    return w1, b1, w2, b2

In [106]:
def ReLU(z):
    return np.maximum(z, 0)

def Softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def derv_ReLU(z):
    return z > 0

In [107]:
# arr is a 2d numpy array, assign the maximum value in each row to 1 and the rest to 0
def max_to_one(arr, m, n):
    for i in range(0, m):
        d = np.argmax(arr[i])
        for j in range(0, n):
            if(d == j):
                arr[i][j] = 1
            else:
                arr[i][j] = 0
    return arr

In [108]:
def forward_prop(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = Softmax(z2)
    a2 = a2.T
    #find the number of rows in a2
    m = a2.shape[0]
    #print(a2)
    a2 = max_to_one(a2, m, 30)
    # print(a2)
    # print(a2.shape)
    return z1, a1, z2, a2

def back_prop(w1, b1, w2, b2, x, y, z1, a1, z2, a2):
    m = y.size
    dz2 = (a2 - y).T
    dw2 = 1 / m * (dz2.dot(a1.T))
    db2 = 1 / m * np.sum(dz2)
    dz1 = w2.T.dot(dz2) * derv_ReLU(z1)
    dw1 = 1 / m * (dz1.dot(x.T))
    db1 = 1 / m * np.sum(dz1)
    return dw1, db1, dw2, db2

def update_var(w1, b1, w2, b2, dw1, db1, dw2, db2, learning_rate):
    w1 = w1 - learning_rate * dw1
    b1 = b1 - learning_rate * db1
    w2 = w2 - learning_rate * dw2
    b2 = b2 - learning_rate * db2
    return w1, b1, w2, b2

In [109]:
def accuracy(y_pred, y):
    return np.mean(y_pred == y)

def grad_descent(x, y, alpha, w1, b1, w2, b2):
    z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x)
    dw1, db1, dw2, db2 = back_prop(w1, b1, w2, b2, x, y, z1, a1, z2, a2)
    w1, b1, w2, b2 = update_var(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
    return w1, b1, w2, b2, a2

In [116]:
from sklearn.utils import shuffle
batch_size = 32
n_example = 2790
batches = int(n_example/batch_size)
epochs = 100
w1, b1, w2, b2 = init_var()
# print(w1)
acc = np.empty(0)
for epoch in range(epochs):
    i = shuffle(range(n_example))
    for b in range(batches):
        x_batch = X_train[i[b*batch_size:(b+1)*batch_size]]
        y_batch = y_train[i[b*batch_size:(b+1)*batch_size]]
        w1, b1, w2, b2, a2 = grad_descent(x_batch.T, y_batch, 0.1, w1, b1, w2, b2)
        acc = np.append(acc, accuracy(a2, y_batch))
    print("Iteration: ", epoch+1)
    print("Train Accuracy: ", np.mean(acc))
# print(w1)

Iteration:  1
Train Accuracy:  0.938146551724138
Iteration:  2
Train Accuracy:  0.938374042145594
Iteration:  3
Train Accuracy:  0.9387452107279693
Iteration:  4
Train Accuracy:  0.939218151340996
Iteration:  5
Train Accuracy:  0.9394971264367815
Iteration:  6
Train Accuracy:  0.9397190293742017
Iteration:  7
Train Accuracy:  0.9398535851122057
Iteration:  8
Train Accuracy:  0.9399739583333333
Iteration:  9
Train Accuracy:  0.9400768944231588
Iteration:  10
Train Accuracy:  0.9401520593869732
Iteration:  11
Train Accuracy:  0.9402222657610589
Iteration:  12
Train Accuracy:  0.9402787755427842
Iteration:  13
Train Accuracy:  0.9403321175950486
Iteration:  14
Train Accuracy:  0.9403761288998356
Iteration:  15
Train Accuracy:  0.9404150702426566
Iteration:  16
Train Accuracy:  0.9404498922413793
Iteration:  17
Train Accuracy:  0.9404820261437908
Iteration:  18
Train Accuracy:  0.9405105896126011
Iteration:  19
Train Accuracy:  0.9405361464004839
Iteration:  20
Train Accuracy:  0.940557950

In [111]:
n_example_t = 310
batches = int(n_example_t/batch_size)
acc_test = np.empty(0)
i = shuffle(range(n_example_t))
for b in range(batches):
    x_batch_t = X_test[i[b*batch_size:(b+1)*batch_size]]
    y_batch_t = y_test[i[b*batch_size:(b+1)*batch_size]]
    z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x_batch_t.T)
    acc_test = np.append(acc_test, accuracy(a2, y_batch_t))
print("Test Accuracy: ", np.mean(acc_test))

Test Accuracy:  0.9414351851851852


In [8]:
from temp_app import *
from rainfall_api import *

atemp, ah = sense()
rain = get_rain()

ModuleNotFoundError: No module named 'board'

In [117]:
#atemp, ah, rain = 35, 70, 110
l=[]
l.append(atemp)
l.append(ah)
l.append(rain)
#create a numpy arrray with atemp, ah and rain
predictcrop = np.array(l)
predictcrop = predictcrop.reshape(1, 3)
print(predictcrop)
print(predictcrop.shape)

[[ 35  70 110]]
(1, 3)


In [113]:
# Putting the names of crop in a single list
crops=['Black gram', 'Chickpea', 'Coconut', 'Coffee', 'Cotton', 'Ground Nut', 'Jute', 'Kidney Beans', 'Lentil', 'Moth Beans', 'Mung Bean', 'Peas', 'Pigeon Peas', 'Rubber', 'Sugarcane', 'Tea', 'Tobacco', 'apple', 'banana', 'grapes', 'maize', 'mango', 'millet', 'muskmelon', 'orange', 'papaya', 'pomegranate', 'rice', 'watermelon', 'wheat']
print(len(crops))

30


In [114]:
def pred1(wa1, ba1, wa2, ba2, xa):
    za1 = wa1.dot(xa) + ba1
    aa1 = ReLU(za1)
    za2 = wa2.dot(aa1) + ba2
    aa2 = Softmax(za2)
    aa2 = aa2.T
    return aa2

In [118]:
#print(w1)
# print(b1.shape)
# print(w2.shape)
# print(b2.shape)
# print(predictcrop.shape)
# x1 = w1
# x2 = b1
# x3 = w2
# x4 = b2
print(predictcrop)
a2 = pred1(w1, b1, w2, b2, predictcrop.T)
#print(a2.shape)
print(a2)
#find mean of each column of a2
#a2 = np.mean(a2, axis=0)
#find maximum in a2
# d = np.argmax(a2[0])
# #find the index of the second largest number in a2
# d2 = np.argpartition(a2.flatten(), -2)[-2]
# print("The crop recommended is ",crops[d])
# print(d)
# print("The second crop recommended is ",crops[d2])
# print(d2)
# print(a2)
thres = 0.04
df = np.argwhere(a2 > thres)
# print(df)
print("Recommended crops are: ")
if not df.size:
    print("None suited.")
for i in df:
    print(crops[i[1]])

[[ 35  70 110]]
[[0.03579922 0.03552312 0.0313387  0.03573633 0.04543665 0.02612702
  0.02640023 0.02321325 0.02691745 0.05055361 0.0215469  0.03753383
  0.03267875 0.05020502 0.02058027 0.03265478 0.04091816 0.02626577
  0.03376599 0.02464515 0.03836859 0.02974075 0.02544817 0.02920799
  0.04338035 0.05141546 0.04240186 0.03370489 0.02425643 0.02423534]]
Recommended crops are: 
Cotton
Moth Beans
Rubber
Tobacco
orange
papaya
pomegranate
